# Knapsack con restriccion en el grafo:

Imaginemos ahora que tenemos que resolver en nuestra computadora un qbit solo puede interactuar con otros $n$ qubits. Matematicamente, decimos que se da una interaccion si en en la funcion objetivo, el coeficiente que multiplica a $x_ix_j$ es distinto de $0$.
Por convencion al decir que el limite es $n$ no se contabiliza la interaccion de $x_i$ con el mismo

### Relacionar variables:

La forma de resolver este problema, es relacionando variables. Si ponemos una penalizacion a

$$(x_i^1 + x_i^2 - x_i^1x_i^2)^2$$

Esto hara que $x_i^1 = x_i^2$. A partir de ahora podemos tratar a estas dos variables como una unica variable. Notemos que tanto $x_i^1$ como $x_i^2$ se pueden relacionar con $n-1$ variables, asi que en su conjunto se pueden relacionar con $2n - 2$ variables.

#

Si $n > 2$, entonces $2n - 2 > n$ y ganamos al menos una variable mas.

#

Si estas $2n - 2$ variables no alcanzan, podemos relacionar $x_i^2$ con otra variable $x_i^3$, teniendo ahora $3$ variables relacionadas, y $3n - 4$ conexiones libres:

#

Entre $x_i^1$ y $x_i^3$ tienen $2n -2$ conexiones, y $x_i^2$ tiene $n - 2$ conexiones mas.

#

Siguiendo este proceso, correlacionando $k$ variables con este metodo, se llegan a:

$$C_k = 2n - 2 + (k - 2)(n - 2) = kn - (2k + 2) = C_{k-1} + (n - 2)$$

conexiones. Si $n > 2$ entonces $C_{k} > C_{k-1}$, por lo al aumentar las variables correlacionadas, podemos aumentar arbitrariamente las conexiones posibles.

### Ejemplo problema de Knapsack:

Vamos a resolver el problema de Knapscak clasico, con la restriccion de que el maximo de conexiones posibles es $n = 5$

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
from functions_dwave import *
warnings.filterwarnings('ignore')

In [ ]:
# Creamos una instancia de problema knapsack para trabajar
profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
data = dict({'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
connections = 5 # Maximo posible de conexiones
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print("-------------------------------------")

In [ ]:
# Generacion de q_objetivo
num_slack = int(np.floor(np.log2(max_weight))) + 1
q_objetivo = np.diag(np.r_[profits, np.zeros(num_slack)])
print(q_objetivo)

#### Hasta ahora solo tenemos una matriz diagonal, no gastamos conexiones. Como veremos a continuacion, Q_restriccion si va a generar un problema

Primero, veamos cuantes conexiones agregar por variable. $vect$ tiene potencialmente tantas entradas no nulas como su dimension. Cada variable se relacionara asi con $d - 1$ variables, si $d$ es la dimension de $vect$.

#

Para ver cuantas variables relacionar por cada variable original, debemos encontrar un $k$ tal que:

$$kn - (2k + 2) \geq d - 1$$
$$k \geq \frac{d + 1}{n - 2}$$

Tomamos asi $k$ como:

$$k = \left\lfloor \frac{d + 1}{n - 2} \right\rfloor + 1$$

Recordemos que $n$ es el numero maximo de conexiones

In [ ]:
# Generacion de q_restriccion:
pesosSlacks = (2 ** (np.arange(num_slack - 1))).astype(float)
pesosSlacks = np.r_[pesosSlacks, max_weight - sum(pesosSlacks)] # revisar esto.
vect = np.r_[weights, pesosSlacks]
q_restriccion = np.outer(vect, vect) - 2 * max_weight * np.diag(vect)


print(q_restriccion)

In [ ]:
variables = len(vect)
qbits_relations = int(np.floor((variables + 1)/(connections - 2))) + 1
print(qbits_relations)

# La matriz para la relacion de cada qbit tendra una diagonal principal de 2 y -1's en las secundarias.
# Construimos una de esta matriz, de qbits_relations x qbits_relations y hacemos un producto de Kronnecker:

q_relation_unitaria = 2 * np.diag(np.ones(qbits_relations)) - np.diag(np.ones(qbits_relations - 1), 1) - np.diag(np.ones(qbits_relations - 1), -1)
print(q_relation_unitaria)

# Repetimos el patron, una vez por cada variable
q_relations = np.kron(np.identity(variables), q_relation_unitaria)

### Redimensionamiento de matrices:

Ahora las matrices $Q_{objetivo}$ y $Q_{restriccion}$ no estan bien dimensionadas. Necesitamos ampliar ambas en $k$.

#

Ampliar $Q_{objetivo}$ es relativamente facil, debemos poner los mismos elementos en la diagonal, dejando ahora un espacio de $k$ ceros entre un elemento original y otro:

In [40]:
# Ampliacion de la matriz Q_objetivo:
profits_expanded = profits
new_nums = np.zeros(qbits_relations + (qbits_relations-1)*(qbits_relations-1))
new_nums[::qbits_relations] = profits_expanded
new_nums = np.r_[new_nums, np.array(np.zeros(qbits_relations-1))]
print(new_nums)

# Agregamos ademas qbits_relations * num_slacks ceros:
q_objetivo = np.diag(np.r_[new_nums, np.zeros(qbits_relations * num_slack)])

# Si esta bien, al menos deberian tener las mismas dimensiones:
print(np.shape(q_objetivo))
print(np.shape(q_relations))



[18.  0.  0.  0.  0. 15.  0.  0.  0.  0. 10.  0.  0.  0.  0. 10.  0.  0.
  0.  0. 18.  0.  0.  0.  0.]
(55, 55)
(55, 55)


Ampliar $Q_{restriccion}$ es mas dificil, pero se puede realizar de la siguiente forma: